In [1]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from numpy import asarray
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

import pandas as pd

In [2]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [3]:
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # make a prediction on the test set
        yhat = model.predict(X_test)
        # round probabilities to class labels
        yhat = yhat.round()
        # calculate accuracy
        acc = accuracy_score(y_test, yhat)
        # store result
        print('>%.3f' % acc)
        results.append(acc)
    return results

In [4]:
# read data
df = pd.read_excel("menu_data.xlsx")
df.head()

,메뉴,소스,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,8,"['4', '5']",0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
1,11,"['5', '14']",0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
2,18,"['5', '14']",0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,8,"['3', '5']",0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,5,"['1', '3', '5']",1,0,1,0,1,0,0,0,0,0,0,0,0,0,0


In [24]:
X = df[['메뉴']]
X = X.to_numpy()
X, X.shape

(array([[ 8],
        [11],
        [18],
        ...,
        [ 3],
        [ 3],
        [ 3]], dtype=int64),
 (1325, 1))

In [23]:
y = df.iloc[:, 2:]
y = y.to_numpy()
y, y.shape

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 1]], dtype=int64),
 (1325, 15))

In [25]:
# evaluate model
results = evaluate_model(X, y)

5/5 [==============================] - 0s 1ms/step
>0.038
5/5 [==============================] - 0s 5ms/step
>0.000
5/5 [==============================] - 0s 3ms/step
>0.038
5/5 [==============================] - 0s 3ms/step
>0.053
5/5 [==============================] - 0s 4ms/step
>0.038
5/5 [==============================] - 0s 3ms/step
>0.023
5/5 [==============================] - 0s 5ms/step
>0.030
5/5 [==============================] - 0s 5ms/step
>0.038
5/5 [==============================] - 0s 4ms/step
>0.030
5/5 [==============================] - 0s 5ms/step
>0.053
5/5 [==============================] - 0s 4ms/step
>0.008
5/5 [==============================] - 0s 3ms/step
>0.045
5/5 [==============================] - 0s 5ms/step
>0.053
5/5 [==============================] - 0s 6ms/step
>0.038
5/5 [==============================] - 0s 3ms/step
>0.038
5/5 [==============================] - 0s 5ms/step
>0.045
5/5 [==============================] - 0s 5ms/step
>0.008
5/5 [=========

In [26]:
# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

Accuracy: 0.034 (0.015)


In [27]:
n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, y, verbose=0, epochs=100)

In [37]:
# make a prediction for new data
newX = [[7]]
yhat = model.predict(newX)
print('Predicted: %s' % yhat[0])

1/1 [==============================] - 0s 62ms/step
Predicted: [0.3959518  0.0688234  0.23889664 0.15023279 0.3257888  0.13149914
 0.06834185 0.02368045 0.17169833 0.10352503 0.0600313  0.11344989
 0.34044144 0.11641325 0.02624689]
